<a href="https://colab.research.google.com/github/alexandergribenchenko/DS_LATAM_Test/blob/main/LATAM_N_02_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

## B. Dataframe raw

In [18]:
path_github = 'https://raw.githubusercontent.com/alexandergribenchenko/DS_LATAM_Test/main/dataset_SCL.csv'             

In [19]:
df_raw = pd.read_csv(path_github, dtype=object)
df_raw

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,22,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,25,12,2017,Lunes,I,JetSmart SPA,Santiago,Lima
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,27,12,2017,Miercoles,I,JetSmart SPA,Santiago,Lima
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,29,12,2017,Viernes,I,JetSmart SPA,Santiago,Lima


# 03. Transformadores

## 03.01. FeatureSelector

In [61]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_FeatureSelector):
        self.feature_names = params_FeatureSelector['feature_names'] 
    
    def fit(self, X, y = None):
        return self 
    
    def transform(self, X, y = None):
      X_output = X[self.feature_names]
      return X_output

## 03.02. DateType

In [63]:
class DateType(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_FeatureSelector):
        self.cols_dates = params_DateType['cols_dates'] 
    
    def fit(self, X, y = None):
        return self 
    
    def transform(self, X, y = None):
      X_output = X.copy()
      for col in self.cols_dates:
        X_output[col] = pd.to_datetime(X_output[col])
      return X_output

## 03.03. TemporadaAlta

In [72]:
class TemporadaAlta(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_TemporadaAlta):
        self.columna_fecha = params_TemporadaAlta['columna_fecha'] 
    
    def fit(self, X, y = None):
        return self

    def fun_temporada_alta(self, fecha):
      mes = fecha.month
      dia = fecha.day
      if (mes == 12 and dia >= 15) or \
      (mes in [1, 2]) or \
      (mes == 3 and dia <= 3) or \
      (mes == 7 and (dia >= 15 and dia <= 31)) or \
      (mes == 9 and (dia >= 11 and dia <= 30)):
          return 1
      else:
          return 0
    
    def transform(self, X, y = None):
      X_output = X.copy()
      # X_output[self.columna_fecha] = pd.to_datetime(X_output[self.columna_fecha])
      X_output['temporada_alta'] = X_output[self.columna_fecha].apply(self.fun_temporada_alta)
      return X_output

## 03.04. DifMin

In [90]:
class DifMin(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_DifMin):
        self.empty = params_DifMin 
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
      X_output = X.copy()
      X_output['dif_min'] = (X_output['Fecha-O']-X_output['Fecha-I'])/pd.Timedelta(minutes=1)
      return X_output

## 03.05. Atraso15

In [102]:
class Atraso15(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_Atraso15):
        self.empty = params_Atraso15 
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
      X_output = X.copy()
      X_output['atraso_15'] = (X_output['dif_min'] > 15).astype(int)
      return X_output

## 03.05. PeriodoDia

In [113]:
class PeriodoDia(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_PeriodoDia):
        self.columna_fecha = params_PeriodoDia['columna_fecha'] 
    
    def fit(self, X, y = None):
        return self

    def fun_periodo_dia(self, fecha):
      hora = fecha.hour
      if hora >= 5 and hora <= 11:
          return 'mañana'
      elif hora >= 12 and hora <= 18:
          return 'tarde'
      else:
          return 'noche'
    
    def transform(self, X, y = None):
      X_output = X.copy()
      X_output['periodo_dia'] = X_output[self.columna_fecha].apply(self.fun_periodo_dia)
      return X_output

## 03.05. OrderOutput

In [119]:
class OrderOutput(BaseEstimator, TransformerMixin):
    
    def __init__(self, params_OrderOutput):
        self.feature_names = params_OrderOutput['feature_names'] 
    
    def fit(self, X, y = None):
        return self 
    
    def transform(self, X, y = None):
      X_output = X[self.feature_names]
      return X_output

# 04. Pipeline

## 04.01. Parametros

In [122]:
params_FeatureSelector = {}
params_FeatureSelector['feature_names']= ['Fecha-I', 
                                          'Des-I',
                                          'Emp-I', 
                                          'TIPOVUELO', 
                                          'Fecha-O',
                                         ]

params_DateType = {}
params_DateType['cols_dates']= ['Fecha-I', 'Fecha-O']

params_TemporadaAlta = {}
params_TemporadaAlta['columna_fecha']= 'Fecha-I'

params_DifMin = {}

params_Atraso15 = params_DifMin

params_PeriodoDia = params_TemporadaAlta

params_OrderOutput = {}
params_OrderOutput['feature_names']= ['temporada_alta',
                                      'dif_min',
                                      'atraso_15',	
                                      'periodo_dia'
                                         ]

In [123]:
Transformer_FeatureSelector = FeatureSelector(params_FeatureSelector)
Transformer_DateType = DateType(params_DateType)
Transformer_TemporadaAlta = TemporadaAlta(params_TemporadaAlta)
Transformer_DifMin = DifMin(params_DifMin)
Transformer_Atraso15 = Atraso15(params_Atraso15)
Transformer_PeriodoDia = PeriodoDia(params_PeriodoDia)
Transformer_OrderOutput = OrderOutput(params_OrderOutput)

In [124]:
pipeline_data_LATAM = Pipeline(steps=[('NameFeatureSelector', Transformer_FeatureSelector),
                              ('NameDateType', Transformer_DateType),
                              ('NameTemporadaAlta', Transformer_TemporadaAlta),
                              ('NameDifMin', Transformer_DifMin),
                              ('NameDateAtraso15', Transformer_Atraso15),
                              ('NamePeriodoDia', Transformer_PeriodoDia),
                              ('NameOrderOutput', Transformer_OrderOutput)
                              ])

In [125]:
df = pipeline_data_LATAM.transform(df_raw)
df

,temporada_alta,dif_min,atraso_15,periodo_dia
0,1,3.0,0,noche
1,1,9.0,0,noche
2,1,9.0,0,noche
3,1,3.0,0,noche
4,1,-2.0,0,noche
...,...,...,...,...
68201,1,46.0,1,tarde
68202,1,16.0,1,tarde
68203,1,40.0,1,tarde
68204,1,13.0,0,tarde
